In [ ]:
API_KEY=""
START_TIMESTAMP=1737666000 #Thursday, January 23rd, 2025
END_TIMESTAMP=1738789200 #Wednesday, February 5th, 2025

In [ ]:
import os
import time
import json
import requests
from tqdm import tqdm

In [ ]:
#get summoner_id of challengers in a region
def get_challengers(region_short):
  link = f'https://{region_short}.api.riotgames.com/tft/league/v1/challenger?queue=RANKED_TFT&api_key={API_KEY}'
  response = requests.get(link)
  if(response.status_code == 200):
    challengers = response.json()['entries']
    challengers_puuid = [x['summonerId'] for x in challengers]
    return challengers_puuid
  else:
    return response.status_code

In [ ]:
#get summoner_id of grandmasters in a region
def get_grandmaster(region_short):
  link = f'https://{region_short}.api.riotgames.com/tft/league/v1/grandmaster?queue=RANKED_TFT&api_key={API_KEY}'
  response = requests.get(link)
  if(response.status_code == 200):
    grandmasters = response.json()['entries']
    grandmasters_puuid = [x['summonerId'] for x in grandmasters]
    return grandmasters_puuid
  else:
    return response.status_code

In [ ]:
#convert summoner_id to puid for match data pull
def summonerId_to_puuid(summonerId, region_short):
  link = f'https://{region_short}.api.riotgames.com/tft/league/v1/entries/by-summoner/{summonerId}?api_key={API_KEY}'
  response = requests.get(link)
  if(response.status_code == 200):
    return response.json()[0]['puuid']
  else:
    return response.status_code

In [ ]:
#get match ids for a puid
def get_matchId(puuid, count, region_long):
  link = f'https://{region_long}.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start=0&endTime={END_TIMESTAMP}&startTime={START_TIMESTAMP}&count={count}&api_key={API_KEY}'
  response = requests.get(link)
  if(response.status_code == 200):
    return response.json()
  else:
    return response.status_code

In [ ]:
def challenger_matches(matchIds, region_short, region_long):
  challengers = get_challengers(region_short) #first get the summoner ids of challengers
  if (type(challengers) != int):
    for challenger in tqdm(challengers):
      time.sleep(1) #pause for api limits
      puuid = summonerId_to_puuid(challenger, region_short) #convert the summoner ids to puids
      if (type(puuid) != int):
        summoner_matchIds = get_matchId(puuid, 10000, region_long) #get the most recent 10,000 matches (returns less given the time range) for a puid of a challenger
        if (type(summoner_matchIds) !=int):
          matchIds.update(summoner_matchIds) #create a list of the match ids
        else:
          print(f'Error getting match ids: {summoner_matchIds} on {puuid}') 
      else:
        print(f'Error converting summonerId to puuid: {puuid} on {challenger}')
  else:
    print(f'Error returning list of challengers: {challengers}')

In [ ]:
#is the same as challenger_matches but gets data for grandmasters instead
def grandmasters_matches(matchIds, region_short, region_long):
  grandmasters = get_grandmaster(region_short)
  if (type(grandmasters) != int):
    for grandmaster in tqdm(grandmasters):
      time.sleep(1)
      puuid = summonerId_to_puuid(grandmaster, region_short)
      if (type(puuid) != int):
        summoner_matchIds = get_matchId(puuid, 10000, region_long)
        if (type(summoner_matchIds) !=int):
          matchIds.update(summoner_matchIds)
        else:
          print(f'Error getting match ids: {summoner_matchIds} on {puuid}')
      else:
        print(f'Error converting summonerId to puuid: {puuid} on {grandmaster}')
  else:
    print(f'Error returning list of challengers: {grandmasters}')

In [ ]:
region_short = 'NA1'
region_long = 'AMERICAS'
matchIds = set()

In [ ]:
challenger_matches(matchIds, region_short, region_long)
grandmasters_matches(matchIds, region_short, region_long)
len(matchIds)

 36%|███▌      | 90/250 [02:26<03:50,  1.44s/it]

Error converting summonerId to puuid: 500 on hR3eJUzt0gHwt-kHP9mnImMkJ5RcNSJNIFebzJf_sSeHUmk


 42%|████▏     | 105/250 [03:09<17:53,  7.40s/it]

Error converting summonerId to puuid: 504 on 0cHOqz_gCG6alVWcMTiDsUMdVgT1O1Ks7Cb5jsrhHw9zCpIM


 81%|████████  | 202/250 [05:38<01:06,  1.40s/it]

...

Error converting summonerId to puuid: 500 on orsLGXmVSxMQNz8EB0ldwTumaU-sG6KUtbi04MzCb2Gc56E


 99%|█████████▉| 494/500 [12:42<00:11,  1.96s/it]

Error converting summonerId to puuid: 504 on 294GyQa_UaSoqyw_Ar3BGJ1rT_uRnjVGcBdfP-aqsqQFUps


100%|██████████| 500/500 [12:52<00:00,  1.54s/it]


18557

In [ ]:
region_short='BR1'
challenger_matches(matchIds, region_short, region_long)
grandmasters_matches(matchIds, region_short, region_long)

100%|██████████| 100/100 [02:50<00:00,  1.70s/it]


In [ ]:
len(matchIds)

23010

In [ ]:
region_short='EUW1'
region_long = 'EUROPE'
challenger_matches(matchIds, region_short, region_long)
grandmasters_matches(matchIds, region_short, region_long)
len(matchIds)

 67%|██████▋   | 134/200 [03:25<01:33,  1.42s/it]

Error converting summonerId to puuid: 500 on I0l7yAvFsTWTWKd0tI6ZvSzlxqgFABQRczJ7fDU_Z8oOj98


 70%|███████   | 141/200 [03:35<01:23,  1.42s/it]

Error converting summonerId to puuid: 500 on tAPBxW6GCVGxxAXGdzwkF4YFWbTJlJMD07gVPhUn7abAME2p9VeLEFc0kQ


 76%|███████▌  | 303/400 [07:15<02:16,  1.41s/it]

Error converting summonerId to puuid: 500 on A56Y17iGdInIayXb-GkZ1Yfn1WJimGjK4UwCqoNF6vXkPWg


 80%|████████  | 322/400 [07:42<01:45,  1.35s/it]

Error converting summonerId to puuid: 500 on aVCKJLSkpgnfUrsODwvynp2ASdphZVDXfUALFnGL1B_h2KobSi69qfti_w


100%|██████████| 400/400 [09:34<00:00,  1.44s/it]


39357

In [ ]:
region_short='KR'
region_long = 'ASIA'
challenger_matches(matchIds, region_short, region_long)
grandmasters_matches(matchIds, region_short, region_long)
len(matchIds)

  0%|          | 1/300 [00:01<08:24,  1.69s/it]

Error converting summonerId to puuid: 403 on YG3XJEC2UjcZJmU4NfsG7ybf7CEU_ERQgXVuSnoJLn2FeFW-GHAdqiO31A


  1%|          | 2/300 [00:04<10:51,  2.19s/it]

Error getting match ids: 400 on DKei458K3d_tZ7amA_5G5A4iSTzb6JW1o2gL0DWl4himWCN2-3UfsAvyodpjg-fA83qnwqVI4aPybw


  1%|          | 3/300 [00:06<10:25,  2.11s/it]

...

Error getting match ids: 400 on tRGlll5o3GSL52azWdX9rXwsQvgN38f4AhbWwaM4dkHte7Enzx0_3JsJyUkq-K7MUN65PubUjcqjgg


  3%|▎         | 10/300 [00:20<10:10,  2.11s/it]

Error getting match ids: 400 on HoBzNwotk1V8_lrlFWCI28dWxLO-LJwZTWLhiPIrcUqvLTVPT1bbWU80GxPZIa44IEbXN6jPcB7I9w


100%|██████████| 600/600 [16:16<00:00,  1.63s/it]


67700

In [ ]:
#no access to china data, unfortunate because they are arguably the best region

In [ ]:
with open('MATCHIDS_PATCH13.4.json', "w") as file:
    json.dump(list(matchIds), file)